#### Mixed Integer Programming (MIP) applied to an Operations Research (OR) problem of procurements / productions:
<br>

In daily operations or decision making processes of supply chain or manufacturing industry, there are scenarios for computations of the input resources to be consumed subject to objectives of maximizing or minimizing a target. The problem settings are to be self-defined by analysts or specialists in which a number of constraints would be proposed, with optimization algorithms, an optimal solution could be found given the variables could be feasibly solved conforming all constraints.

This notebook demonstrates a procurement or a production context, where decisions were to be assigning each of a set of products to one of the sources. If the organiztion was a producer possessing production lines, then the sources would be raw materials suppliers, while if the organization was a wholesaler or distributor, then it would be placing procurement orders to purchase finished goods from sourcing vendors. The objective would be minimizing the total processing time (production, order processing, delivery, etc.), which would bring benefits of cutting production or processing costs with shorter lead time.

<br>
Define the following variables: <br>
$St = Materials\ / \ Stocks$<br>
$Pc = Weekly\ Production\ Capacity$<br>
$Lot = Lotsize\ / \ Minimum\ Order\ Quantity$<br>
$Dem = Demand\ Quantity$<br>
$Lt = Lead\ Time\ (weeks)$<br>
$Sc = Supplier\ Total\ Production\ Capacity$<br>
<br>
Decision and objective variables: <br>
$Pt = Total\ Processing\ Time$<br>
$V{ij} = I\ (product(i)\ assigned\ to\ source(j))$<br>
<br>
Subject to the following constraints for products i's and sources j's:<br>
$\sum_{j=1}^{J} V{ij} = 1 \ $  (i.e. each product allocated to only 1 source)<br>
$ST{i} + PC{i} \ * \ PT{i} \ >= DEM{i} \ $ (i.e. demanded quautity should be fulfilled)<br>
$PC{i} \ + \ PT{i} \ >= LOT{i} \ $ (i.e. each batch of production should be fulfilled the lotsize requirement)<br>
$PT{i} \ >= LT{i} \ $ (i.e. total processing time should exceed the production lead time)<br>
$\sum_{i=1}^{I} V{ij} \ * PC{ij} \ * \ PT{i} \ <= SC{j} \ $  (i.e. all production quantities should not exceed total capacity for each source)<br>
<br>
Objective function:<br>
$minimize\ ( \sum_{i=1}^{I} \sum_{j=1}^{J} PT{ij} )$<br>
<br>

The simulator() function was used to generate hypothetic data given specification on some upper and lower limits to imitate a real situation. Then, the models() function tried to optimize the mixed integer programming model with the constraints and objective funciton. Sometimes, there might be no feasible solution, or sometimes multiple solutions returned in the pool of solutions. In this example, 1 optimal solution was given, and reviewing the results, 3 types of products particularly had long processing periods due to low weekly capacity of resources allocated. Perhaps, they are likely high-end products with long development cycle. Though, this was from simulation perspective testing the implementation of the model while the real data might not behave like this. 


In [1]:
!pip install mip

     |████████████████████████████████| 48.0 MB 1.7 MB/s 


In [2]:
import mip
import numpy as np
import pandas as pd
import json
import seaborn
import itertools

In [ ]:
## Example Problem Sets and Solution in MIP ###
## https://python-mip.readthedocs.io/en/latest/examples.html ##

In [ ]:
#### Variable List ####
# Material / Stock Reserved = st
# Supplier Capacity in Total = sc
# Mininum Order Qty (Lotsize) = lot
# Demand Qty (from targeted customer) = dem
# Processing Time = "objective", pt1, pt2, pt3...
# LeadTime = lt
# Production Capacity = pc

In [3]:
## Data simulations
import random
random.seed(1)

def simulator(n_targets, n_sources):
    st = list()  ## stock / reserved inventory
    pc = list()  ## weekly productivity
    lo = list()  ## lotsize
    de = list()  ## demands
    lt = list()  ## lead time
    sc = list()  ## source-specific total capacity

    up_st, low_st = 1000, 0
    up_pc, low_pc = 3000, 100
    up_lo, low_lo = 10000, 2000
    up_de, low_de = 40000, 2000
    up_lt, low_lt = 16, 6
    up_sc, low_sc = 300, 100

    for n in range(n_targets):
        st.append(random.randint(low_st, up_st))
        pc.append(random.randint(low_pc, up_pc))
        lo.append(random.randint(low_lo, up_lo))
        de.append(random.randint(low_de, up_de))
        lt.append(random.randint(low_lt, up_lt))

    for j in range(n_sources):
        sc.append(random.randint(low_sc, up_sc) * 1000)

    data_job = pd.DataFrame({'item':list(range(n_targets)),
                             'stock':st, 
                             'capacity':pc,
                             'lotsize':lo,
                             'demands':de,
                             'lead_time':lt
                             })
    data_source = pd.DataFrame({'vendor':list(range(n_sources)),
                                'capacity':sc
                                })
    return data_job, data_source

In [7]:
## objective:   min(sum(pt(i))
## constraint 1:  st(i) + pc(i) * pt(i) >= de(i)
## constraint 2:  pc(i) * pt(i) >= lo(i) 
## constraint 3:  pt(i) >= lt(i)
## constraint 4:  sum(pc(i,j)) <= sc(j) for j in len(n_sources)

def models(jobs, sources):

    st = jobs.iloc[:,1].values.tolist()
    pc = jobs.iloc[:,2].values.tolist()
    lot = jobs.iloc[:,3].values.tolist()
    dem = jobs.iloc[:,4].values.tolist()
    lt = jobs.iloc[:,5].values.tolist()
    s = sources.iloc[:,0].values.tolist()
    sc = sources.iloc[:,1].values.tolist()

    ## mip model
    model = mip.Model(sense = mip.MINIMIZE, solver_name = mip.CBC)
    ## decision variable
    v = [[model.add_var(name="v({},{})".format(n, m), var_type = mip.BINARY)
          for n in range(sources.shape[0])]
         for m in range(jobs.shape[0])]
    ## outcome variable
    pt = [model.add_var(name="pt({})".format(n), var_type = mip.INTEGER)
          for n in range(jobs.shape[0])]
    ## constraints
    for i in range(len(pt)):
        model.add_constr(st[i] + pc[i] * pt[i] >= dem[i])
        model.add_constr(pc[i] * pt[i] >= lot[i])
        model.add_constr(pt[i] >= lt[i])
        model.add_constr( mip.xsum(v[i][j] for j in range(sources.shape[0])) == 1 )
    for j in range(sources.shape[0]):
        model.add_constr( mip.xsum(pc[i] * pt[i] for i in range(len(pc)) if v[i] == s[j]) <= sc[j] )
    ## objective
    model.objective = mip.xsum(pt[i] for i in range(len(pt)))
    ## optimize
    model_state = model.optimize()
    
    return model, model_state

In [5]:
data_job, data_source = simulator(n_targets = 10, n_sources = 2)

In [8]:
model, model_state = models(data_job, data_source)

In [11]:
model_state == mip.OptimizationStatus.OPTIMAL

True

In [12]:
model_state

<OptimizationStatus.OPTIMAL: 0>

In [14]:
print("Number of solution: " + str(model.num_solutions))
print("================================================")
for sol in range(model.num_solutions):
    print("Optimal Solution " + str(sol + 1) + " :")
    for v in range(len(model.vars)):
        if "v" in model.vars[v].name:
            if model.vars[v].xi(sol) == 1:
                print("Product " + str(int(model.vars[v].name[-2:-1]) + 1) + " allocated at (or procured from) Source " + str(int(model.vars[v].name[2]) + 1))
        elif "pt" in model.vars[v].name:
            print("Product " + str(int(model.vars[v].name[-2:-1]) + 1) + " total processing time = " + str(int(model.vars[v].xi(sol))) + " weeks")

Number of solution: 1
Optimal Solution 1 :
Product 1 allocated at (or procured from) Source 2
Product 2 allocated at (or procured from) Source 1
Product 3 allocated at (or procured from) Source 1
Product 4 allocated at (or procured from) Source 1
Product 5 allocated at (or procured from) Source 2
Product 6 allocated at (or procured from) Source 2
Product 7 allocated at (or procured from) Source 1
Product 8 allocated at (or procured from) Source 1
Product 9 allocated at (or procured from) Source 2
Product 10 allocated at (or procured from) Source 1
Product 1 total processing time = 10 weeks
Product 2 total processing time = 15 weeks
Product 3 total processing time = 10 weeks
Product 4 total processing time = 126 weeks
Product 5 total processing time = 284 weeks
Product 6 total processing time = 11 weeks
Product 7 total processing time = 197 weeks
Product 8 total processing time = 12 weeks
Product 9 total processing time = 73 weeks
Product 10 total processing time = 11 weeks


In [9]:
## simulate 10 products to be produced/procured
data_job

,item,stock,capacity,lotsize,demands,lead_time
0,0,137,2431,8942,6135,10
1,1,120,2129,8233,31457,13
2,2,667,1654,8461,15759,7
3,3,499,216,9318,27546,12
4,4,622,108,7700,31188,10
5,5,738,1037,6842,8699,11
6,6,31,191,2208,37482,6
7,7,961,1661,7623,16195,12
8,8,743,218,6322,16528,13
9,9,961,2130,6529,17275,11


In [10]:
## simulate 2 vendors/suppliers
data_source

,vendor,capacity
0,0,159000
1,1,273000
